# Simulate reservoir routine
***

**Author:** Chus Casado Rodríguez<br>
**Date:** 20-06-2024<br>

**Introduction:**<br>
This code simulates all the reservoirs included both in GloFASv4 and ResOpsUS according to the reservoir routine defined in the configuration file (attribute `simulation>model`).

The inflow time series is taken from GloFASv4 simulations, and the initial storage from the observed records.

>Note. The `Shrestha` reservoir routine requires a time series of water demand as input. Since that time series is not available, the code creates a fake demand by a transformation of the input time series.

**To do:**<br>
* [ ] When using the `Lisflood` model, some parameters could be estimated according to the records:
```Python
# storage limits
Vn_adj = get_normal_value(obs.storage) 
Vn = 1 * Vn_adj
Vtot = reservoirs.loc[grand_id, 'CAP'] * 1e6
Vf = Vtot - .2 * (Vtot - Vn_adj)

# outflow limits
if obs.outflow.isnull().all():
    Qn = reservoirs.loc[grand_id, 'normq_adj']
else:
    Qn = get_normal_value(obs.outflow) 
if obs.inflow.isnull().all():
    Qmin, Qnd = Qmin, Qnd = reservoirs.loc[grand_id, ['minq', 'ndq']]
else:
    Q100 = return_period(obs.inflow, T=100)
    Qnd = .3 * Q100
    Qmin = obs.inflow.groupby(obs.index.year).quantile(.05).mean()
Qmin = np.min([Qmin, Qn])
```

**Ideas:**<br>


In [1]:
import sys
sys.path.append('../../src/')
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from pathlib import Path
import yaml

from lisfloodreservoirs.models import get_model
from lisfloodreservoirs.utils.metrics import KGEmod
from lisfloodreservoirs.utils.utils import get_normal_value, return_period

## Configuration

In [2]:
with open('config_linear_2var.yml', 'r', encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

### Paths
# PATH_GLOFAS = Path(cfg['paths']['GloFAS'])
PATH_RESOPS = Path(cfg['paths']['ResOpsUS'])
# PATH_GRAND = Path(cfg['paths']['GRanD'])

### Reservoir model
MODEL = cfg['simulation']['model'].lower()
MODEL_CFG = cfg['simulation'].get('config', {})

# results will be saved in this path
PATH_OUT = Path(f'{MODEL}/default')
PATH_OUT.mkdir(parents=True, exist_ok=True)
print(f'Results will be saved in {PATH_OUT}')

Results will be saved in linear\default


In [3]:
variables = ['inflow', 'storage', 'outflow']

## Data

### Attributes

In [4]:
# import all tables of attributes
path_attrs = PATH_RESOPS / 'attributes'
try:
    attributes = pd.concat([pd.read_csv(file, index_col='GRAND_ID') for file in path_attrs.glob('*.csv')], axis=1, join='inner')
except Exception as e:
    raise ValueError('ERROR while reading attribute tables: {}'.format(e)) from e
print(f'{attributes.shape[0]} reservoirs in the attribute tables')

# keep only reservoirs with all observed variables
mask = pd.concat([attributes[var.upper()] == 1 for var in variables], axis=1).all(axis=1)
attributes = attributes[mask]
print('{0} reservoirs include observed timeseris for all variables: {1}'.format(attributes.shape[0],
                                                                                *variables))

118 reservoirs in the attribute tables
63 reservoirs include observed timeseris for all variables: inflow


#### Time series

Time series of reservoirs simulated in GloFAS, as the GloFAS simulated inflow will be used as the forcing of the reservoir module.

In [5]:
path_ts = PATH_RESOPS / 'time_series' / 'csv'
timeseries = {}
for grand_id in tqdm(attributes.index, desc='reading time series'):
    file = path_ts / f'{grand_id}.csv'
    if file.is_file():
        ts = pd.read_csv(file, parse_dates=True, index_col='date')
    else:
        print(f"File {file} doesn't exist")
        continue
    # select columns associated with variables of interest
    select_columns = [col for col in ts.columns if col.split('_')[0] in variables]
    ts = ts[select_columns]
    if not ts.columns.str.contains('glofas').any():
        print(f'{grand_id} does not contain GloFAS simulated time series')
        continue
    # invert normalization
    capacity = attributes.loc[grand_id, 'CAP_MCM'] * 1e6
    ts *= capacity
    ts.iloc[:, ts.columns.str.contains('inflow')] /= (24 * 3600)
    ts.iloc[:, ts.columns.str.contains('outflow')] /= (24 * 3600)
    # save time series
    timeseries[grand_id] = ts
    
print(f'{len(timeseries)} reservoirs with timeseries')

reading time series:   0%|          | 0/63 [00:00<?, ?it/s]

41 does not contain GloFAS simulated time series
182 does not contain GloFAS simulated time series
185 does not contain GloFAS simulated time series
600 does not contain GloFAS simulated time series
59 reservoirs with timeseries


In [6]:
# path_ts = PATH_RESOPS / 'time_series' / 'csv'
# timeseries = {}
# # for grand_id in tqdm(attributes['glofas'].index, desc='reading time series'):
# for file in tqdm(list(path_ts.glob('*.csv')), desc='reading time series'):
#     grand_id = int(file.stem)
#     # read time series
#     if grand_id in attributes['glofas'].index:
#         ts = pd.read_csv(file, parse_dates=True, index_col='date')
#     else:
#         continue
#     # select columns associated with variables of interest
#     select_columns = [col for col in ts.columns if col.split('_')[0] in variables]
#     ts = ts[select_columns]
    
#     # invert normalization
#     capacity = attributes['grand'].loc[grand_id, 'CAP_MCM'] * 1e6
#     ts *= capacity
#     ts.iloc[:, ts.columns.str.contains('inflow')] /= (24 * 3600)
#     ts.iloc[:, ts.columns.str.contains('outflow')] /= (24 * 3600)
#     # save time series
#     timeseries[grand_id] = ts
    
# print(f'{len(timeseries)} reservoirs with timeseries')

## Reservoir routine
### Simulate all reservoirs

In [7]:
# GloFAS reservoir
for grand_id, ts in tqdm(timeseries.items(), desc='simulating reservoir'): # 146 #302 #273 #236 #227
    
    # file where the simulation results will be saved
    file_out = PATH_OUT / f'{grand_id:03}_performance.csv'
    if file_out.is_file():
        print(f'The file {file_out} already exists.')
        continue
        
    ## TIME SERIES
    try:
        # observed time series
        obs = ts[ts.columns.intersection(variables)].copy()
        obs[obs < 0] = np.nan
        # GloFAS simulated time series
        glofas = ts[[f'{var}_glofas' for var in variables]]
        glofas.columns = variables
    except Exception as e:
        print(f'ERROR. The time series of reservoir {grand_id} could not be set up\n', e)
        continue

    # storage limits (m3)
    Vtot, Vmin = attributes.loc[grand_id, ['CAP_MCM', 'Vmin']]
    Vtot *= 1e6
    Vmin *= Vtot
    # outflow limits (m3/s)
    Qmin = attributes.loc[grand_id, 'Qmin']
    
    if MODEL.lower() == 'linear':
        # residence time: number of days required to fill the reservoir with the mean inflow
        # if 'inflow' in obs.columns:
        if (~obs.inflow.isnull()).sum() > 365 * 4:
            T = Vtot / (obs.inflow.mean() * 24 * 3600)
        # elif 'outflow' in obs.columns:
        elif (~obs.outflow.isnull()).sum() > 365 * 4:
            T = Vtot / (obs.outflow.mean() * 24 * 3600)
        # elif 'inflow' in glofas.columns:
        if (~glofas.inflow.isnull()).sum() > 365 * 4:
            T = Vtot / (glofas.inflow.mean() * 24 * 3600)
        else:
            print(f'Reservoir {grand_id} does not have neither inflow nor ouflow observation, so the residence time cannot be estimated')
            continue
        kwargs = {'Vmin': Vmin, 'Vtot': Vtot, 'Qmin': Qmin, 'T': T} 
    
    elif MODEL.lower() == 'lisflood':
        # storage limits (m3)
        Vn, Vn_adj, Vf = attributes.loc[grand_id, ['Vn', 'Vn_adj', 'Vf']] * Vtot
        # outflow limits (m3)
        Qn, Qf = attributes.loc[grand_id, ['Qn_adj', 'Qf']]
        # keyword arguments
        kwargs = {'Vmin': Vmin, 'Vn': Vn, 'Vn_adj': Vn_adj, 'Vf': Vf, 'Vtot': Vtot, 'Qmin': Qmin, 'Qn': Qn, 'Qf': Qf}
    
    elif MODEL.lower() == 'hanazaki':
        # storage limits (m3)
        Vf = obs.storage[start:].quantile(.75)
        Ve = Vtot - .2 * (Vtot - Vf)
        Vmin = .5 * Vf
        # outflow limits
        if (~obs.inflow.isnull()).sum() > 365 * 4:
            inflow = obs.inflow
        else:
            inflow = glofas.inflow
        # inflow = obs.inflow if 'inflow' in obs.columns else glofas.inflow            
        Qn = inflow.mean()
        Q100 = return_period(inflow, T=100)
        Qf = .3 * Q100
        # catchment area (m2)
        A = attributes.loc[grand_id, 'CATCH_SKM'] * 1e6
        # keyword arguments
        kwargs = {'Vmin': Vmin, 'Vf': Vf, 'Ve': Ve, 'Vtot': Vtot, 'Qn': Qn, 'Qf': Qf, 'A': A}
    
    elif MODEL.lower() == 'shrestha':
        # create a fake demand
        # inflow = obs.inflow if 'inflow' in obs.columns else glofas.inflow
        inflow = glofas.inflow
        demand = .8 * inflow + np.random.normal(loc=0, scale=inflow.std() * .5, size=inflow.shape)
        demand[demand < 0] = 0
        # normal filling
        storage = obs.storage if 'storage' in obs.columns else glofas.storage
        gamma = obs.median() / Vtot
        # keyword arguments
        kwargs = {'Vmin': Vmin, 'Vtot': Vtot, 'Qmin': Qmin, 'avg_inflow': inflow.mean(), 'avg_demand': demand.mean(), 'gamma': gamma}
        MODEL_CFG = {'demand': demand}
        
    # # export default parameters
    # with open(PATH_OUT / f'{grand_id:03}_default_parameters.yml', 'w') as file:
    #     yaml.dump(kwargs, file)
    
    # declare the reservoir
    res = get_model(MODEL, **kwargs)

    # simulate the reservoir
    if 'storage' in obs:
        Vo = obs.loc[obs.storage.first_valid_index(), 'storage']
    else:
        Vo = None
    sim = res.simulate(glofas.inflow, Vo, **MODEL_CFG)

    # analyse simulation
    performance = pd.DataFrame(index=['KGE', 'alpha', 'beta', 'rho'], columns=obs.columns)
    for var in performance.columns:
        try:
            performance[var] = KGEmod(obs[var], sim[var])
        except:
            continue
    performance.to_csv(file_out, float_format='%.3f')
    res.scatter(sim,
                obs,
                norm=False,
                title=f'grand_id: {grand_id}',
                save=PATH_OUT / f'{grand_id}_scatter_default.jpg',
               )
    
    res.lineplot({'GloFAS': glofas, 
                  'sim': sim},
                 obs,
                 figsize=(12, 6),
                 save=PATH_OUT / f'{grand_id}_line_default.jpg',
               )

simulating reservoir:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/13878 [00:00<?, ?it/s]

  0%|          | 0/13878 [00:00<?, ?it/s]

  0%|          | 0/12739 [00:00<?, ?it/s]

  0%|          | 0/12781 [00:00<?, ?it/s]

...

  0%|          | 0/13878 [00:00<?, ?it/s]

  0%|          | 0/12326 [00:00<?, ?it/s]

  0%|          | 0/13878 [00:00<?, ?it/s]

  0%|          | 0/12934 [00:00<?, ?it/s]

  0%|          | 0/13514 [00:00<?, ?it/s]